# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

## CONV3D - Final Model

This project involves building a 3D Convolutional Neural Network (CNN) to correctly recognize hand gestures by a user to control a smart TV.

The objective of this projects is to build a hand gesture recognition model that can be hosted on a camera installed in a smart TV that can understand 5 gestures. Namely, leftwards hand movement to go to previous channel, rightward hand movement to go to next channel, upward hand movement to increase the volume, downward hand movement to decrease the volume and a palm gesture to pause playing the video.

In [150]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [152]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, folder names for training and validation are read. We also set the batch_size here. We set the batch size in such a way that we are able to use the GPU in full capacity. We keep increasing the batch size until the machine throws an error.

In [153]:
train_doc = np.random.permutation(open('C:\\Users\\adithya\\OneDrive\\Desktop\\dl\\Project_data\\train.csv').readlines())
val_doc = np.random.permutation(open('C:\\Users\\adithya\\OneDrive\\Desktop\\dl\\Project_data\\val.csv').readlines())
batch_size = 64

## CONV3D Generator

In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames.

In [154]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,18,60,60,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(60,60))
                    image = image - np.min(image)/ (np.max(image) - np.min(image))
                   
                    batch_data[folder,idx,:,:,0] = image[:,:,0] 
                    batch_data[folder,idx,:,:,1] = image[:,:,1] 
                    batch_data[folder,idx,:,:,2] = image[:,:,2] 

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,18,60,60,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(60,60))
                    image = image - np.min(image)/ (np.max(image) - np.min(image))
                                      
                    batch_data[folder,idx,:,:,0] = image[:,:,0] 
                    batch_data[folder,idx,:,:,1] = image[:,:,1] 
                    batch_data[folder,idx,:,:,2] = image[:,:,2] 


                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [155]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:\\Users\\adithya\\OneDrive\\Desktop\\dl\\Project_data\\train'
val_path = 'C:\\Users\\adithya\\OneDrive\\Desktop\\dl\\Project_data\\val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
A model is created using different functionalities that Keras provides.

In [156]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers

model = Sequential()
model.add(Conv3D(32, (2,2,2), strides=(1,1,1), padding='same', input_shape=(18,60,60,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(64, (2,2,2), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(128, (2,2,2), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (2,2,2), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

Now compiling the model and looking at the summary to find out the number of parameters of the model. The number of parameters of the model is important since the model has to be light enough to be hosted on a webcam itself.

In [157]:
# sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
optimiser = optimizers.Adam(lr=0.0002)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 18, 60, 60, 32)    800       
_________________________________________________________________
batch_normalization_41 (Batc (None, 18, 60, 60, 32)    128       
_________________________________________________________________
activation_43 (Activation)   (None, 18, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 30, 30, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 9, 30, 30, 64)     16448     
_________________________________________________________________
batch_normalization_42 (Batc (None, 9, 30, 30, 64)     256       
_________________________________________________________________
activation_44 (Activation)   (None, 9, 30, 30, 64)   

Creating the train_generator and the val_generator which will be used in .fit_generator.

In [158]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [159]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

C:\ProgramData\Anaconda3\envs\nn1\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [160]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Fitting the model and saving checkpoints.

In [161]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  C:\Users\adithya\OneDrive\Desktop\dl\Project_data\val ; batch size = 64
Source path =  C:\Users\adithya\OneDrive\Desktop\dl\Project_data\train ; batch size = 64


C:\ProgramData\Anaconda3\envs\nn1\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  
C:\ProgramData\Anaconda3\envs\nn1\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
C:\ProgramData\Anaconda3\envs\nn1\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
C:\ProgramData\Anaconda3\envs\nn1\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.froma

11/11 [==============================] - 78s 7s/step - loss: 2.0019 - categorical_accuracy: 0.2941 - val_loss: 1.8085 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-11-0221_12_03.930404/model-00001-2.02692-0.29412-1.80845-0.27000.h5
Epoch 2/30
11/11 [==============================] - 65s 6s/step - loss: 1.3421 - categorical_accuracy: 0.3982 - val_loss: 2.1317 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-11-0221_12_03.930404/model-00002-1.34874-0.39819-2.13167-0.23000.h5
Epoch 3/30
11/11 [==============================] - 64s 6s/step - loss: 1.1509 - categorical_accuracy: 0.5038 - val_loss: 2.9973 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-11-0221_12_03.930404/model-00003-1.15176-0.50377-2.99727-0.23000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-05.
Epoch 4/30
11/11 [==============================] - 63s 6s/step - loss: 0.9905 - categorical_acc

11/11 [==============================] - 58s 5s/step - loss: 0.3510 - categorical_accuracy: 0.8718 - val_loss: 0.6198 - val_categorical_accuracy: 0.8400

Epoch 00029: saving model to model_init_2020-11-0221_12_03.930404/model-00029-0.35022-0.87179-0.61978-0.84000.h5
Epoch 30/30
11/11 [==============================] - 58s 5s/step - loss: 0.3329 - categorical_accuracy: 0.8763 - val_loss: 0.6557 - val_categorical_accuracy: 0.8400

Epoch 00030: saving model to model_init_2020-11-0221_12_03.930404/model-00030-0.33798-0.87632-0.65575-0.84000.h5


### Model accuracy
After 30 epochs, the model results in a 'validation categorical accuracy' of 84%. This can be still be improved by fine tuning batch size and number of epochs. But we consider this to be the best model from the different experiments we tried.